**Dharmi Malde
MDS202333**

Write a your own LU solver, based on Gaussian elimination with partial pivoting, in Python.
The input is a square invertible matrix A and a right hand side vector b (both randomly
generated). The output should be L, U and the (row) permutaion matrix and the solution to
the problem Ax = b. Once you are sure that the code is working properly you should experiment
with various sizes and note the time taken. Prepare comparison tables (one for factorization
and the other for the solution, i.e., forward and backword substitution) that shows the time
needed for your algorithm and also for the SciPy’s LU solver. For each system you should also
note down how far were you from the actual solution.


**To be precise, here are some guidelines.**

*   There is a built-in LU solver in SciPy. However, you should write your own without calling
any readily available routine from any of the Python libraries.
*   Use the NumPy random library to generate an n × n real matrix for various values of
n, like n = 10, 50, 100, 400, 800, 1000, . . . etc. (take at least 5 different values, more the
better)


*   Implement the Gaussian elimination with partial pivoting algorithm. Do maintain an
array that keeps track of pivots. In case you reach a situation where there is no (nonzero)
pivot available in that column then you should exit the program, saying “unable to find
nonzero pivot”.
*   The ideal output of the program should be the permutation matrix P, the unit lower
triangular L and the upper triangular U. Of course, you don’t want to print these matrices
individually.


*   Measure the time taken just for the factorization (don’t count matrix initialization, verification etc.) and separately for substituions.


*   Prepare the following tables: matrix norm of PA − LU and the vector norm of Ax0 − b
for both the solvers. If the norm is zero then you are sure that there were no precision
errors. Compare these values for the in-built solver with yours.

**To be precise, here are some guidelines.**

*   You should write an helper functions.py” which will have all the auxiliary functions for
your implementation.


*   Document each of your functions well (at least one line for each).


*   Write the "results.ipynb" file, which will import the "helper functions.py" file and display the results.

*   use an understandable naming convention for your functions (don’t use one alphabet for
defining functions and variables).
*    The pdf of the results notebook should be named "your name roll no.pdf"









In [ ]:
import numpy as np
from scipy.linalg import solve
import time as t
from scipy.linalg import lu
import pandas as pd

In [ ]:
def LUSolver(A,b):
  n = len(A)
  L = np.eye(n)
  U = np.copy(A)
  P = np.eye(n)

  # FInding LU decomposition
  time_1 = t.time()
  for i in range(n):
    # Partial pivoting
    max_index = np.argmax(abs(U[i:, i])) + i

    if max_index != i:
      U[[i, max_index]] = U[[max_index, i]]
      P[[i, max_index]] = P[[max_index, i]]
      for j in range(0,i):
        L[i,j], L[max_index,j] = L[max_index,j],L[i,j]

    for j in range(i+1, n):
      L[j, i] = U[j, i] / U[i, i]
      U[j, i:] = U[j, i:] - L[j, i] * U[i, i:]
  time_2 = t.time()
  # Solving Ax = b
  new_b = np.dot(P, b)
  # Solving Ly = b
  y = np.zeros(n)
  y[0] = b[0] / L[0, 0]
  for i in range(1, n):
      y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]

  # Solving Ux = y
  x = np.zeros(n)
  x[-1] = y[-1] / U[-1, -1]
  for i in range(n-2, -1, -1):
      x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]

  return L,U,P,x,time_1,time_2

In [ ]:
# Generate random matrix list A and b
n = [10,50,100,400,800,1000]
matrix_list = []
for _ in n:
    A = np.random.rand(_, _)
    b = np.random.rand(_)
    matrix_list.append((A, b))

In [ ]:
matrix_norm = []
vector_norm = []
matrix_norm2 = []
vector_norm2 = []
time_LU = []
time_solve = []
time_scipy = []
for i in matrix_list:
  ud_L,ud_U,ud_P,ud_x, time1, time2 = LUSolver(i[0],i[1])
  time3 = t.time()
  x = solve(i[0], i[1])
  time4 = t.time()
  P, L, U = lu(i[0])
  mat = np.dot(ud_L,ud_U) - np.dot(ud_P,i[0])
  vec = np.dot(i[0],ud_x) - i[1]
  mat1 = np.dot(L,U) - np.dot(P,i[0])
  vec1 = np.dot(i[0],x) - i[1]
  time_LU.append(time2 - time1)
  time_solve.append(time3 - time2)
  time_scipy.append(time4 - time3)
  matrix_norm.append(np.linalg.norm(mat, ord='fro'))
  vector_norm.append(np.linalg.norm(vec))
  matrix_norm2.append(np.linalg.norm(mat, ord='fro'))
  vector_norm2.append(np.linalg.norm(vec))

data = {"n":n,
        "Matrix norm": matrix_norm,
        "vector norm": vector_norm,
        "Scipy Matrix norm": matrix_norm2,
        "Sci vector norm": vector_norm2,
        "Decomposition Time": time_LU,
        "Substitution Time": time_solve,
        "Scipy Time": time_scipy
        }



In [ ]:
df = pd.DataFrame(data)
df

,n,Matrix norm,vector norm,Scipy Matrix norm,Sci vector norm,Decomposition Time,Substitution Time,Scipy Time
0,10,5.523399e-16,1.499554,5.523399e-16,1.499554,0.000932,0.000374,0.000473
1,50,9.200648e-15,2.872253,9.200648e-15,2.872253,0.017771,0.000423,0.000364
2,100,2.818024e-14,4.182409,2.818024e-14,4.182409,0.046380,0.005011,0.002566
3,400,3.389271e-13,8.230700,3.389271e-13,8.230700,0.695996,0.003721,0.022765
4,800,1.254739e-12,11.521372,1.254739e-12,11.521372,2.343682,0.007431,0.040609
5,1000,1.945924e-12,13.561912,1.945924e-12,13.561912,5.793091,0.010206,0.090745
